### Task Schedule:
1. 訓練模型,調整參數(預計使用lgb，速度較快)(更:使用catboost,效果較好)
2. 嘗試使用不同模型,做Ensamble(blending, stacking)
3. Anomaly detection

### 注意事項:
1. 因為test data和train data時間不相關,在驗證時採取前60天訓練61~90天驗證,但仍需小心時間差異造成的影響
2. Anomaly detection: 看這類的模型能不能取代boosting(似乎是不行，盜刷數據並沒有那麼Anomaly）,但可以嘗試將Anomaly結果當成新feature

### <font color=green>Results:</font>

#### Catboost:
    * FE1~4,catboost訓練 validation:0.5, LB:0.55
    * FE1,3,4 validation:0.5313149, LB:0.6(th=0.37), default parameter

#### LGB:
    * 不做處理,直接丟lgb訓練 leaderboard score:0.45
    # 585
    # 593
    # 不同的thread不同的結果 乾==

## 讀取,轉換字串成可以訓練的資料

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import math

import lightgbm as lgb
from lightgbm.sklearn import LGBMClassifier
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from catboost import CatBoostClassifier, Pool

from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn import svm
from sklearn.ensemble  import  GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

%matplotlib inline
data_path = '../../data'

random_seed = 20
import json
path = '../../code/para_dict/data_list.json'
with open(path,'r',encoding='utf-8') as f:
    para = json.loads(f.read())
    
data_list= para['data_list_FE_AN7']
delete_list = para['delete_list_overfit1']

## 除掉一些可能會overfit,distribution不同,受時間影響大的feature

In [2]:
def load_data(data_list):
    data=[]
    for d in data_list:
        x = pd.read_csv('../../data/preprocess/{}'.format(d))
        x_null = x.isnull().sum()
        print('\n',d,x.shape)
        print("Null columns:\n",x_null[x_null>0])

        if (d=='FE_data1.csv') or (d=='FE_data2.csv'):
            x.fillna(value=-1,inplace=True)
        
        if d[:8]=='FE_data9':
            if d!='FE_data9_raw.csv':
                x = x.drop(columns=['bacno_shift1','bacno_shiftm1'])
        data.append(x)

    all_data = pd.concat(data,axis=1)
    del data
    all_data_numsum = all_data.isnull().sum()
    print('ALL data shape:',all_data.shape)
    print('ALL data null:')
    print(all_data_numsum[all_data_numsum>0])
    return all_data

def transform_data(all_data,category_list,bool_list):
    for c in category_list:
        if all_data[c].dtypes == 'float64':
            all_data[c] = all_data[c].astype('int')
        all_data[c]=all_data[c].astype('category')

    for c in all_data.columns[all_data.dtypes==bool]:
        all_data[c]=all_data[c].map({True:1,False:0})

    for c in bool_list:
        if c in all_data.columns:
            all_data[c]=all_data[c].map({'True':1,'False':0,'-1':-1})
    
    return all_data

all_data = load_data(data_list)


 raw_data.csv (1943452, 23)
Null columns:
 fraud_ind    421665
dtype: int64

 FE_data1.csv (1943452, 56)
Null columns:
 cano_conam_skew      92612
cano_conam_kurt     155720
cano_conam_var       38678
bacno_locdt_skew     58303
bacno_locdt_kurt    101191
cano_locdt_skew      92612
cano_locdt_kurt     155720
dtype: int64

 FE_data2.csv (1943452, 30)
Null columns:
 Series([], dtype: int64)

 FE_data2_2.csv (1943452, 31)
Null columns:
 Series([], dtype: int64)

 FE_data3.csv (1943452, 9)
Null columns:
 Series([], dtype: int64)

 FE_data4.csv (1943452, 4)
Null columns:
 Series([], dtype: int64)

 FE_data4_2.csv (1943452, 3)
Null columns:
 Series([], dtype: int64)

 FE_data5.csv (1943452, 4)
Null columns:
 Series([], dtype: int64)

 FE_data6.csv (1943452, 17)
Null columns:
 Series([], dtype: int64)

 FE_data8.csv (1943452, 11)
Null columns:
 Series([], dtype: int64)

 FE_data9_3.csv (1943452, 20)
Null columns:
 Series([], dtype: int64)

 FE_data9_4.csv (1943452, 10)
Null columns:
 Series([

In [3]:
import sys
print("Memory size of "+str(sys.getsizeof(all_data))+ " bytes")

Memory size of 5158706539 bytes


In [4]:

category_list=['csmcu','hcefg','stscd','scity','stocn','mcc',\
               'acqic','mchno','etymd','contp','locdt_week']
bool_list= ['cano_lastlocdt2_shift1','cano_lastlocdt2_shiftm1','bacno_stscd_equal2_shift1','bacno_stscd_equal2_shiftm1',\
            'bacno_ecfg_equal1_shift1','bacno_ecfg_equal1_shiftm1']

# main
all_data = transform_data(all_data,category_list,bool_list)


In [5]:
## 切三種不同的訓練集驗證
X_train1 = all_data[all_data['locdt']<=60].drop(columns=delete_list)
y_train1 = all_data[all_data['locdt']<=60]['fraud_ind']
X_test1 = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)].drop(columns=delete_list)
y_test1 = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)]['fraud_ind']

# test_data_txkey = all_data[all_data['locdt']>90]['txkey'].copy().values
# X_train_all = all_data[all_data['locdt']<=90].drop(columns=delete_list) 
# y_train_all = all_data[all_data['locdt']<=90]['fraud_ind'] 
# X_test_all = all_data[all_data['locdt']>90].drop(columns=delete_list) 
# y_test_all = all_data[all_data['locdt']>90]['fraud_ind'] 

## Train on catboost
* https://catboost.ai/docs/concepts/python-reference_parameters-list.html
* 研究有哪些可以用的function

In [6]:
categorical_features_indices = np.where(X_train1.columns.isin(category_list))[0]
print(X_train1.dtypes[categorical_features_indices])

acqic         category
contp         category
csmcu         category
etymd         category
hcefg         category
mcc           category
mchno         category
scity         category
stocn         category
stscd         category
locdt_week    category
dtype: object


In [7]:
param_cat={
"loss_function":"Logloss",
"eval_metric":"F1",

"iterations":6000,
"scale_pos_weight":1,
"target_border":0.5,
"thread_count":1,
"task_type":"GPU",
"devices":"0-2",
"verbose":20,

"learning_rate":0.03,
"l2_leaf_reg":1.5928,
"depth":15,
"max_leaves":35,
"bagging_temperature":0.0520,
"random_strength":10,
"one_hot_max_size":200,
"grow_policy":"Lossguide"
}

# 'random_strength':1,
# 'depth':6,
# 'grow_policy':'Lossguide',
# 'min_data_in_leaf':1,
# # 'max_leaves':31,
# 'boosting_type':'Ordered',
# 'gpu_ram_part':0.5,
    
# 將binary的資料轉成category


In [8]:
model = CatBoostClassifier(**param_cat)

model.fit(
    X_train1, y_train1,
    cat_features=categorical_features_indices,    
    eval_set=(X_test1, y_test1),
    early_stopping_rounds=1000,
#     use_best_model=True,
    silent=False,
#     plot=True,
)
print('Model is fitted: ' + str(model.is_fitted()))
print('Model params:')
print(model.get_params())

0:	learn: 0.6891841	test: 0.4710761	best: 0.4710761 (0)	total: 70.1ms	remaining: 7m
1:	learn: 0.6919169	test: 0.4713101	best: 0.4713101 (1)	total: 175ms	remaining: 8m 44s
2:	learn: 0.6877606	test: 0.4690129	best: 0.4713101 (1)	total: 212ms	remaining: 7m 4s
3:	learn: 0.6914696	test: 0.4652859	best: 0.4713101 (1)	total: 255ms	remaining: 6m 21s
4:	learn: 0.6947044	test: 0.4713630	best: 0.4713630 (4)	total: 379ms	remaining: 7m 34s
5:	learn: 0.6921183	test: 0.4638374	best: 0.4713630 (4)	total: 418ms	remaining: 6m 57s
6:	learn: 0.6967678	test: 0.4652278	best: 0.4713630 (4)	total: 455ms	remaining: 6m 29s
7:	learn: 0.6938714	test: 0.4659892	best: 0.4713630 (4)	total: 500ms	remaining: 6m 14s
8:	learn: 0.6974704	test: 0.4717766	best: 0.4717766 (8)	total: 623ms	remaining: 6m 54s
9:	learn: 0.6981725	test: 0.4744752	best: 0.4744752 (9)	total: 661ms	remaining: 6m 36s
10:	learn: 0.6972882	test: 0.4727622	best: 0.4744752 (9)	total: 699ms	remaining: 6m 20s
11:	learn: 0.6977382	test: 0.4712671	best: 0.4

95:	learn: 0.7746960	test: 0.5477294	best: 0.5481516 (93)	total: 5.5s	remaining: 5m 38s
96:	learn: 0.7754443	test: 0.5472637	best: 0.5481516 (93)	total: 5.54s	remaining: 5m 37s
97:	learn: 0.7761840	test: 0.5491329	best: 0.5491329 (97)	total: 5.65s	remaining: 5m 40s
98:	learn: 0.7769056	test: 0.5514213	best: 0.5514213 (98)	total: 5.68s	remaining: 5m 38s
99:	learn: 0.7774966	test: 0.5524249	best: 0.5524249 (99)	total: 5.72s	remaining: 5m 37s
100:	learn: 0.7782056	test: 0.5526316	best: 0.5526316 (100)	total: 5.75s	remaining: 5m 35s
101:	learn: 0.7786411	test: 0.5529113	best: 0.5529113 (101)	total: 5.79s	remaining: 5m 34s
102:	learn: 0.7795071	test: 0.5521699	best: 0.5529113 (101)	total: 5.92s	remaining: 5m 39s
103:	learn: 0.7805342	test: 0.5525678	best: 0.5529113 (101)	total: 5.96s	remaining: 5m 38s
104:	learn: 0.7809607	test: 0.5540166	best: 0.5540166 (104)	total: 6s	remaining: 5m 36s
105:	learn: 0.7813494	test: 0.5536580	best: 0.5540166 (104)	total: 6.03s	remaining: 5m 35s
106:	learn: 0

187:	learn: 0.8140276	test: 0.5869344	best: 0.5869344 (187)	total: 10.7s	remaining: 5m 30s
188:	learn: 0.8142821	test: 0.5871332	best: 0.5871332 (188)	total: 10.7s	remaining: 5m 30s
189:	learn: 0.8143220	test: 0.5871477	best: 0.5871477 (189)	total: 10.9s	remaining: 5m 31s
190:	learn: 0.8148175	test: 0.5876916	best: 0.5876916 (190)	total: 10.9s	remaining: 5m 31s
191:	learn: 0.8152147	test: 0.5874394	best: 0.5876916 (190)	total: 10.9s	remaining: 5m 30s
192:	learn: 0.8153868	test: 0.5878904	best: 0.5878904 (192)	total: 11s	remaining: 5m 31s
193:	learn: 0.8157678	test: 0.5882486	best: 0.5882486 (193)	total: 11.1s	remaining: 5m 32s
194:	learn: 0.8159954	test: 0.5884741	best: 0.5884741 (194)	total: 11.1s	remaining: 5m 31s
195:	learn: 0.8163732	test: 0.5886860	best: 0.5886860 (195)	total: 11.2s	remaining: 5m 30s
196:	learn: 0.8168250	test: 0.5888714	best: 0.5888714 (196)	total: 11.2s	remaining: 5m 30s
197:	learn: 0.8172603	test: 0.5893622	best: 0.5893622 (197)	total: 11.3s	remaining: 5m 31s
1

278:	learn: 0.8380644	test: 0.5978552	best: 0.5980326 (276)	total: 16s	remaining: 5m 28s
279:	learn: 0.8381462	test: 0.5976986	best: 0.5980326 (276)	total: 16s	remaining: 5m 27s
280:	learn: 0.8382871	test: 0.5979220	best: 0.5980326 (276)	total: 16.1s	remaining: 5m 27s
281:	learn: 0.8385030	test: 0.5981455	best: 0.5981455 (281)	total: 16.1s	remaining: 5m 26s
282:	learn: 0.8386256	test: 0.5981455	best: 0.5981455 (281)	total: 16.2s	remaining: 5m 26s
283:	learn: 0.8386665	test: 0.5977885	best: 0.5981455 (281)	total: 16.2s	remaining: 5m 27s
284:	learn: 0.8386256	test: 0.5977885	best: 0.5981455 (281)	total: 16.3s	remaining: 5m 26s
285:	learn: 0.8386665	test: 0.5981455	best: 0.5981455 (281)	total: 16.4s	remaining: 5m 27s
286:	learn: 0.8387892	test: 0.5980118	best: 0.5981455 (281)	total: 16.4s	remaining: 5m 26s
287:	learn: 0.8390117	test: 0.5980348	best: 0.5981455 (281)	total: 16.5s	remaining: 5m 26s
288:	learn: 0.8390481	test: 0.5976781	best: 0.5981455 (281)	total: 16.6s	remaining: 5m 27s
289

371:	learn: 0.8542524	test: 0.6049656	best: 0.6049656 (371)	total: 21.5s	remaining: 5m 24s
372:	learn: 0.8546013	test: 0.6048780	best: 0.6049656 (371)	total: 21.5s	remaining: 5m 24s
373:	learn: 0.8545613	test: 0.6047904	best: 0.6049656 (371)	total: 21.6s	remaining: 5m 25s
374:	learn: 0.8547504	test: 0.6049451	best: 0.6049656 (371)	total: 21.7s	remaining: 5m 25s
375:	learn: 0.8549602	test: 0.6053215	best: 0.6053215 (375)	total: 21.7s	remaining: 5m 24s
376:	learn: 0.8550603	test: 0.6061143	best: 0.6061143 (376)	total: 21.8s	remaining: 5m 24s
377:	learn: 0.8551200	test: 0.6060472	best: 0.6061143 (376)	total: 21.8s	remaining: 5m 24s
378:	learn: 0.8552898	test: 0.6062486	best: 0.6062486 (378)	total: 21.9s	remaining: 5m 24s
379:	learn: 0.8553595	test: 0.6064030	best: 0.6064030 (379)	total: 22s	remaining: 5m 25s
380:	learn: 0.8558675	test: 0.6065174	best: 0.6065174 (380)	total: 22s	remaining: 5m 25s
381:	learn: 0.8558574	test: 0.6069332	best: 0.6069332 (381)	total: 22.1s	remaining: 5m 24s
382

...

5625:	learn: 1.0000000	test: 0.6504470	best: 0.6509786 (5558)	total: 5m 7s	remaining: 20.4s
5626:	learn: 1.0000000	test: 0.6505206	best: 0.6509786 (5558)	total: 5m 7s	remaining: 20.4s
5627:	learn: 1.0000000	test: 0.6505206	best: 0.6509786 (5558)	total: 5m 7s	remaining: 20.3s
5628:	learn: 1.0000000	test: 0.6503679	best: 0.6509786 (5558)	total: 5m 7s	remaining: 20.3s
5629:	learn: 1.0000000	test: 0.6505206	best: 0.6509786 (5558)	total: 5m 7s	remaining: 20.2s
5630:	learn: 1.0000000	test: 0.6505206	best: 0.6509786 (5558)	total: 5m 7s	remaining: 20.2s
5631:	learn: 1.0000000	test: 0.6505206	best: 0.6509786 (5558)	total: 5m 7s	remaining: 20.1s
5632:	learn: 1.0000000	test: 0.6504470	best: 0.6509786 (5558)	total: 5m 7s	remaining: 20s
5633:	learn: 1.0000000	test: 0.6502943	best: 0.6509786 (5558)	total: 5m 7s	remaining: 20s
5634:	learn: 1.0000000	test: 0.6504470	best: 0.6509786 (5558)	total: 5m 7s	remaining: 19.9s
5635:	learn: 1.0000000	test: 0.6504470	best: 0.6509786 (5558)	total: 5m 7s	remaining

5716:	learn: 1.0000000	test: 0.6495242	best: 0.6515735 (5662)	total: 5m 11s	remaining: 15.4s
5717:	learn: 1.0000000	test: 0.6499773	best: 0.6515735 (5662)	total: 5m 11s	remaining: 15.4s
5718:	learn: 1.0000000	test: 0.6500566	best: 0.6515735 (5662)	total: 5m 11s	remaining: 15.3s
5719:	learn: 1.0000000	test: 0.6502831	best: 0.6515735 (5662)	total: 5m 11s	remaining: 15.3s
5720:	learn: 1.0000000	test: 0.6503568	best: 0.6515735 (5662)	total: 5m 12s	remaining: 15.2s
5721:	learn: 1.0000000	test: 0.6502831	best: 0.6515735 (5662)	total: 5m 12s	remaining: 15.2s
5722:	learn: 1.0000000	test: 0.6503623	best: 0.6515735 (5662)	total: 5m 12s	remaining: 15.1s
5723:	learn: 1.0000000	test: 0.6506679	best: 0.6515735 (5662)	total: 5m 12s	remaining: 15.1s
5724:	learn: 1.0000000	test: 0.6505151	best: 0.6515735 (5662)	total: 5m 12s	remaining: 15s
5725:	learn: 1.0000000	test: 0.6505151	best: 0.6515735 (5662)	total: 5m 12s	remaining: 14.9s
5726:	learn: 1.0000000	test: 0.6506679	best: 0.6515735 (5662)	total: 5m 

5806:	learn: 1.0000000	test: 0.6504249	best: 0.6515735 (5662)	total: 5m 16s	remaining: 10.5s
5807:	learn: 1.0000000	test: 0.6505778	best: 0.6515735 (5662)	total: 5m 16s	remaining: 10.5s
5808:	learn: 1.0000000	test: 0.6505778	best: 0.6515735 (5662)	total: 5m 16s	remaining: 10.4s
5809:	learn: 1.0000000	test: 0.6506516	best: 0.6515735 (5662)	total: 5m 16s	remaining: 10.3s
5810:	learn: 1.0000000	test: 0.6507307	best: 0.6515735 (5662)	total: 5m 16s	remaining: 10.3s
5811:	learn: 1.0000000	test: 0.6507307	best: 0.6515735 (5662)	total: 5m 16s	remaining: 10.2s
5812:	learn: 1.0000000	test: 0.6507307	best: 0.6515735 (5662)	total: 5m 16s	remaining: 10.2s
5813:	learn: 1.0000000	test: 0.6503568	best: 0.6515735 (5662)	total: 5m 16s	remaining: 10.1s
5814:	learn: 1.0000000	test: 0.6506570	best: 0.6515735 (5662)	total: 5m 16s	remaining: 10.1s
5815:	learn: 1.0000000	test: 0.6506516	best: 0.6515735 (5662)	total: 5m 16s	remaining: 10s
5816:	learn: 1.0000000	test: 0.6501246	best: 0.6515735 (5662)	total: 5m 

5896:	learn: 1.0000000	test: 0.6517159	best: 0.6519375 (5864)	total: 5m 20s	remaining: 5.6s
5897:	learn: 1.0000000	test: 0.6516421	best: 0.6519375 (5864)	total: 5m 20s	remaining: 5.55s
5898:	learn: 1.0000000	test: 0.6516421	best: 0.6519375 (5864)	total: 5m 20s	remaining: 5.49s
5899:	learn: 1.0000000	test: 0.6518686	best: 0.6519375 (5864)	total: 5m 20s	remaining: 5.44s
5900:	learn: 1.0000000	test: 0.6517159	best: 0.6519375 (5864)	total: 5m 21s	remaining: 5.38s
5901:	learn: 1.0000000	test: 0.6516421	best: 0.6519375 (5864)	total: 5m 21s	remaining: 5.33s
5902:	learn: 1.0000000	test: 0.6514894	best: 0.6519375 (5864)	total: 5m 21s	remaining: 5.28s
5903:	learn: 1.0000000	test: 0.6517159	best: 0.6519375 (5864)	total: 5m 21s	remaining: 5.22s
5904:	learn: 1.0000000	test: 0.6514894	best: 0.6519375 (5864)	total: 5m 21s	remaining: 5.17s
5905:	learn: 1.0000000	test: 0.6515632	best: 0.6519375 (5864)	total: 5m 21s	remaining: 5.11s
5906:	learn: 1.0000000	test: 0.6517898	best: 0.6519375 (5864)	total: 5m

5985:	learn: 1.0000000	test: 0.6512419	best: 0.6519375 (5864)	total: 5m 25s	remaining: 762ms
5986:	learn: 1.0000000	test: 0.6512419	best: 0.6519375 (5864)	total: 5m 25s	remaining: 707ms
5987:	learn: 1.0000000	test: 0.6512419	best: 0.6519375 (5864)	total: 5m 25s	remaining: 653ms
5988:	learn: 1.0000000	test: 0.6510889	best: 0.6519375 (5864)	total: 5m 25s	remaining: 599ms
5989:	learn: 1.0000000	test: 0.6513897	best: 0.6519375 (5864)	total: 5m 25s	remaining: 544ms
5990:	learn: 1.0000000	test: 0.6515426	best: 0.6519375 (5864)	total: 5m 25s	remaining: 490ms
5991:	learn: 1.0000000	test: 0.6513949	best: 0.6519375 (5864)	total: 5m 26s	remaining: 435ms
5992:	learn: 1.0000000	test: 0.6513158	best: 0.6519375 (5864)	total: 5m 26s	remaining: 381ms
5993:	learn: 1.0000000	test: 0.6510889	best: 0.6519375 (5864)	total: 5m 26s	remaining: 327ms
5994:	learn: 1.0000000	test: 0.6513158	best: 0.6519375 (5864)	total: 5m 26s	remaining: 272ms
5995:	learn: 1.0000000	test: 0.6513949	best: 0.6519375 (5864)	total: 5

In [12]:
print(1)

1


In [10]:
feature_names = X_test1.columns
validation_pool=Pool(X_test1, y_test1,cat_features=categorical_features_indices)

# PredictionValuesChange
feature_importances = model.get_feature_importance(validation_pool)
print('PredictionValuesChange')
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score)) 

PredictionValuesChange
cano_conam_kurt: 2.585303020609506
bacno_max_conam: 2.335157903077442
cano_locdt_skew: 2.2923749227389405
cano_shiftm1: 2.252846217096295
bacno_mchno_mode: 2.2362003113131994
cano_conam_skew: 2.2103790423151763
cano_locdt_kurt: 2.1283962449903333
bacno_mean_conam: 1.8754339196036791
cano_mchno_mode: 1.8234039867859322
bacno_locdt_skew: 1.6347568038462168
bacno_ratio_ecfg: 1.5667747832184868
bacno_locdt_kurt: 1.550947736434755
etymd_ratio: 1.424295820958277
txkey: 1.3967863433980368
pca2_feature_1: 1.3859157286220611
isolationtree_all_feature2: 1.2532070443993073
etymd: 1.2071653168622136
csmcu_ratio_cano: 1.198632360634882
mcc_ratio: 1.145345598809391
cano_acqic_mode: 1.135401360975173
contp_ratio_cano: 1.124925811869243
cano_conam_var: 1.0970160272085245
cano_shift1: 1.0530481508188145
isolationtree_all_feature3: 1.0403731811114787
cano_scale_conam: 1.0396214995644661
stocn_value_counts_cano: 1.0278726128437128
cano_mean_conam: 0.9642442284957572
contp_value_cou

In [ ]:
# LossFunctionChange
feature_importances = model.get_feature_importance(validation_pool,type="LossFunctionChange")
print('LossFunctionChange')
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

In [ ]:
# Shapvalue
feature_importances = model.get_feature_importance(validation_pool,type="ShapValues")
print('Shapvalue')
feature_importances = np.mean(abs(feature_importances),axis=0)
for score, name in sorted(zip(feature_importances, feature_names), reverse=True):
    print('{}: {}'.format(name, score))

In [ ]:
res = model.calc_feature_statistics(X_train1,
                                    y_train1,
#                                     prediction_type='Probability',
#                                     cat_feature_values={6189:0}, 
                                    max_cat_features_on_plot=20,
                                    feature=['isolationtree_all_feature2'],#["mchno"],
                                    plot=True)

In [13]:
print(1)

1


In [ ]:
valid_all_data = all_data[(all_data['locdt']>60) & (all_data['locdt']<=90)]

In [ ]:
tmp_df = valid_all_data.groupby(['bacno'])['fraud_ind'].apply(lambda s: s[s==1].shape[0])
print(tmp_df.value_counts())

In [ ]:
valid_all_data_correct = valid_all_data[valid_all_data['fraud_ind']==y_test1_pred]
print(valid_all_data_correct.shape)
print(valid_all_data.shape)

In [ ]:
valid_all_data['correct'] = valid_all_data['fraud_ind']==y_test1_pred

In [ ]:
tmp_df2 = valid_all_data_correct.groupby(['bacno'])['fraud_ind'].apply(lambda s: s[s==1].shape[0])
print(tmp_df2.value_counts())

In [ ]:
bacno_fraud_count = valid_all_data.groupby('bacno').apply(lambda s:s[s['fraud_ind']==1]['bacno'].shape[0])

In [ ]:
list1=['locdt','cano','etymd','mchno','acqic','mcc','stocn','conam','correct','fraud_ind']

list2=['locdt','cano','cano_lastlocdt2','contp','csmcu','correct','fraud_ind',\
       'flbmk','flg_3dsmk','hcefg','insfg','iterm','mchno','ovrlt','correct','fraud_ind',\
       'scity','stocn','stscd','conam','etymd','mcc','ecfg','correct','fraud_ind']

pd.set_option('display.max_columns', 500)
bacno_10fraud_more_list = bacno_fraud_count[(bacno_fraud_count>=2)].index
pd.set_option('display.max_rows', 1000)

cnt=0
for b in bacno_10fraud_more_list:
    cnt+=1
    if cnt>100:
        break
    print()
    
    print(b,valid_all_data[valid_all_data['bacno']==b]['fraud_ind'].sum())
    print(valid_all_data[valid_all_data['bacno']==b][list2])

In [ ]:
## 理論上th設0.5一定是最好的？

y_test1_pred = model.predict_proba(X_test1,verbose=True)[:,1]
th=0.37

y_test1_pred[y_test1_pred>th]=1
y_test1_pred[y_test1_pred<=th]=0
print('F1 score',f1_score(y_test1, y_test1_pred))

tn, fp, fn, tp = confusion_matrix(y_test1, y_test1_pred).ravel()
print('tn fp fn tp')
print(tn, fp, fn, tp)
print('Percision', tp/(tp+fp))
print('Recall',tp/(tp+fn))

In [ ]:
model = CatBoostClassifier(**param_cat)
model.fit(
    X_train_all, y_train_all,
    cat_features=categorical_features_indices,    
    silent=False
)

In [ ]:
y_test_pred_cat = model.predict_proba(X_test_all)[:,1]

print(X_test_all.index)

th=0.35
y_test_pred_cat[y_test_pred_cat>th]=1
y_test_pred_cat[y_test_pred_cat<=th]=0

## write csv

In [ ]:
import time
print(time.localtime( time.time() ))
a = time.localtime( time.time() )
print(str(a.tm_mon)+str(a.tm_mday))

In [ ]:
result = y_test_pred_cat
test_data_txkey = all_data[all_data['locdt']>90]['txkey'].values

print('{}: prediction positive ratio'.format(result.sum()/result.shape[0]))
print('{}: training positive ratio'.format(y_train_all.sum()/y_train_all.shape[0]))

import time
t_now = time.localtime( time.time() )
t = str(t_now.tm_mon)+str(t_now.tm_mday)+str(t_now.tm_hour)+str(t_now.tm_min)
print('Now:',t)

submit_file_name='submit_cat_AN3_over5_th{}_time{}.csv'.format(th,t)
import csv
with open('../../prediction/{}'.format(submit_file_name),'w') as f:
    writer = csv.writer(f)
    writer.writerow(['txkey','fraud_ind'])
    for i in range(result.shape[0]):
        writer.writerow([test_data_txkey[i], result[i]])
        
# with open('../prediction/log.txt','w') as f:
#     print('{}'.format(submit_file_name),file=f)
#     print('delete_list:\n{}'.format(delete_list),file=f)

In [ ]:
## 用不同seed訓練 降低variance

predictions = []

for i in range(10):
    clf = CatBoostClassifier(learning_rate=0.1, iterations=1000, random_seed=i, logging_level='Silent')
    clf.fit(train_df, labels, cat_features=cat_features_ids)
    predictions.append(clf.predict_proba(test_df)[:,1])
    

prediction = np.mean(predictions, axis=0)

pd.DataFrame(
    {'ID':test_id, 'PredictedProb':prediction}
).to_csv(
    'submission_improved_bagged.csv', index=False
)

## 贝叶斯调参
* http://nohup.cc/article/258/
* https://github.com/fmfn/BayesianOptimization

Results:
{'params': {'bagging_temperature': 1.0,
  'learning_rate': 0.2,
  'reg_lambda': 17.267046492047776},
 'target': 0.5339179149920837}

In [ ]:
# from bayes_opt import BayesianOptimization
# def cat_train(bagging_temperature, reg_lambda, learning_rate):
#     params = {
#         'iterations':2000,
#         'depth':6,
#         'bagging_temperature':bagging_temperature,
#         'reg_lambda':reg_lambda,
#         'learning_rate':learning_rate,
#         'loss_function':'Logloss',
#         'eval_metric':'F1',
#         'random_seed':random_seed,
#         'verbose':30
#     }
 
#     model = CatBoostClassifier(**params)
#     # 评价数据集是验证集，评价指标是AUC
#     model.fit(X_train1, y_train1,\
#               eval_set=(X_test1, y_test1),\
#               cat_features=categorical_features_indices,\
#               early_stopping_rounds=200) 
     
#     print(params)
#     score_max = model.best_score_.get('validation').get('F1')
#     return score_max
 
# cat_opt = BayesianOptimization(cat_train, 
#                            {
#                               'bagging_temperature': (1, 50),  
#                               'reg_lambda': (1, 200),
#                               'learning_rate':(0.05, 0.2)
#                             })
 
# cat_opt.maximize(n_iter=15, init_points=random_seed)
# cat_opt.max

## Permutation importance

In [ ]:
# def permutation_scoring(model,X,y):
#     y_pred = model.predict(X)
#     return f1_score(y, y_pred)

# import eli5
# from eli5.sklearn import PermutationImportance

# print(y_test1.sum())
# print(model.score(X_test1, y_test1))
# perm = PermutationImportance(model, random_state=random_seed,scoring=permutation_scoring,n_iter=50).fit(X_test1, y_test1)
# feature_importance1 = pd.DataFrame({'feature':X_test1.columns.tolist(),'importance':perm.feature_importances_})
# delete_col1 = feature_importance1.iloc[:,0][(feature_importance1['importance'].values)<=0.0000]
# print(delete_col1)

# feature_importance1.sort_values(by=['importance'],inplace=True)
# top_col = feature_importance1
# for i in range(top_col.shape[0]):
#     print(top_col.iloc[i].values)

In [ ]:
# from eli5.catboost import explain_weights_catboost
# explain_weights_catboost(model, vec=None, top=70, importance_type='PredictionValuesChange', feature_names=None, pool=None)

## 將使用者分類來訓練模型:
1. 根據cano的個數分類
2. 根據txkey的個數分類
3. 根據stocn的眾數分類

murmur: 感覺這方法怪怪的,tree模型應該就能涵蓋進去

In [ ]:
## txkey
txkey_qcut_id = pd.qcut(all_data['bacno_txkey_nunique'],3,labels=[0,1,2])

# print(all_data['bacno_txkey_nunique'])
# print(txkey_qcut_id)
models=[]
for i in range(3):
    print(i)

## 不同model

In [ ]:
def th_transform(th,y_test1_pred,y_test1):
    y_test1_pred[y_test1_pred>th]=1
    y_test1_pred[y_test1_pred<=th]=0
    print('F1 score',f1_score(y_test1, y_test1_pred))
    
    return y_test1_pred

In [ ]:
rf = RandomForestClassifier(n_estimators=200,random_state=random_seed,verbose=0,class_weight="balanced", criterion='gini')

rf.fit(X_train1, y_train1) 
y_pred_rf = rf.predict_proba(X_test1)[:,1]
th_transform(0.37,y_pred_rf,y_test1)

In [ ]:
def lgb_f1_score(y_true, y_pred):
    y_pred = np.round(y_pred) # scikits f1 doesn't like probabilities
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    print()
    print('tn, fp, fn, tp')
    print(tn, fp, fn, tp)
    return 'f1', f1_score(y_true, y_pred), True

param_dist_lgb = {
                  'num_leaves':26, 
#                   'max_depth':-1, 
                  'learning_rate':0.1, 
                  'n_estimators':2000,
                  'objective': 'binary',
#                   'subsample': 1, 
#                   'colsample_bytree': 0.5, 
                  'lambda_l1': 10,
                  'lambda_l2': 10,
#                   'min_child_weight': 1,
                  'random_state': random_seed,
                 }
evals_result = {}

lgb_clf = LGBMClassifier(**param_dist_lgb)
lgb_clf.fit(X_train1, y_train1,
        eval_set=[(X_train1, y_train1),(X_test1, y_test1)],
        eval_metric=lgb_f1_score,
        early_stopping_rounds=50,
        verbose=True,
        callbacks=[lgb.record_evaluation(evals_result)]
        )
y_test_pred = lgb_clf.predict(X_test1)
print('F1 score',f1_score(y_test1, y_test_pred))

## Anomaly detection
* one class svm
* isolation tree
* replicator NN
* Kmeans?
* KNN(take too much time)

## 異常偵測
wiki<br>
https://zh.wikipedia.org/wiki/%E5%BC%82%E5%B8%B8%E6%A3%80%E6%B5%8B#cite_note-9

因為盜刷很可能都是outlier，一般的機器學習方法在outlier上表現會很差，因此可以用來解釋為什麼會train不好的原因
http://www.cainiaoxueyuan.com/suanfa/7017.html<br>
https://towardsdatascience.com/outlier-detection-with-isolation-forest-3d190448d45e<br>
https://medium.com/@cyeninesky3/oneclass-svm-%E7%95%B0%E5%B8%B8%E6%AA%A2%E6%B8%AC%E4%BB%BB%E5%8B%99-anomaly-detection-%E7%9A%84%E7%AE%97%E6%B3%95%E7%90%86%E8%A7%A3%E8%88%87%E5%AF%A6%E8%B8%90-cf5f0bbb01c0<br>

isolation tree<br>
https://zhuanlan.zhihu.com/p/25040651
https://scikit-learn.org/stable/auto_examples/ensemble/plot_isolation_forest.html
https://towardsdatascience.com/outlier-detection-with-extended-isolation-forest-1e248a3fe97b

oneclass svm<br>
https://scikit-learn.org/stable/auto_examples/svm/plot_oneclass.html

Replicator NN<br>
https://togaware.com/papers/dawak02.pdf

one class kmeans?<br>
https://ai100-2.cupoy.com/mission/D57

## 製作特徵
XGB, LGB, PCA, Isolation Forest, Kmean距離？, oneclass SVM?
當作新feature

In [ ]:
# import xgboost as xgb
# param_dist_xgb = {'learning_rate':0.01, #默认0.3
#               'n_estimators':1000, #树的个数
# #               'max_depth':5,
# #               'min_child_weight':1,
# #               'gamma':0.2,
# #               'subsample':0.8,
# #               'colsample_bytree':0.8,
# #               'objective': 'binary:logistic', #逻辑回归损失函数
# #               'nthread':4,  #cpu线程数
# #               'scale_pos_weight':1,
#               'seed':random_seed}  #随机种子

# evals_result = {}

# xgb_clf = xgb.XGBClassifier(**param_dist_xgb)
# xgb_clf.fit(X_train, y_train,
#         eval_set=[(X_train, y_train),(X_test, y_test)],
#         eval_metric=lgb_f1_score,
#         early_stopping_rounds=600,
#         verbose=True,
# #         callbacks=[xgb.record_evaluation(evals_result)]
#         )

# print('F1',f1_score(y_test, xgb_clf.predict(X_test)))
# xgb_X_train = xgb_clf.apply(X_train)
# xgb_X_test = xgb_clf.apply(X_test)

## Train on LGB(未調參數)(效果不好)

In [ ]:
def lgb_f1_score(y_true, y_pred):
    y_pred = np.round(y_pred) # scikits f1 doesn't like probabilities
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    print()
    print('tn, fp, fn, tp')
    print(tn, fp, fn, tp)
    return 'f1', f1_score(y_true, y_pred), True

param_dist_lgb = {
#                   'num_leaves':45, 
#                   'max_depth':5, 
                  'learning_rate':0.1, 
                  'n_estimators':2000,
                  'objective': 'binary',
#                   'subsample': 1, 
#                   'colsample_bytree': 0.5, 
#                   'lambda_l1': 0.1,
#                   'lambda_l2': 0,
#                   'min_child_weight': 1,
                  'random_state': random_seed,
                 }
evals_result = {}

lgb_clf = LGBMClassifier(**param_dist_lgb)
lgb_clf.fit(X_train1, y_train1,
        eval_set=[(X_train1, y_train1),(X_test1, y_test1)],
        eval_metric=lgb_f1_score,
        early_stopping_rounds=50,
        verbose=True,
        callbacks=[lgb.record_evaluation(evals_result)]
        )
y_test_pred = lgb_clf.predict(X_test1)

In [ ]:
print('Plotting metrics recorded during training...')
ax = lgb.plot_metric(evals_result, metric='f1')
plt.show()

print('Plotting feature importances...')
ax = lgb.plot_importance(lgb_clf, max_num_features=30)
plt.show()

print('Plotting 4th tree...')  # one tree use categorical feature to split
ax = lgb.plot_tree(lgb_clf, tree_index=3, figsize=(15, 15), show_info=['split_gain'])
plt.show()

print('Plotting 4th tree with graphviz...')
graph = lgb.create_tree_digraph(lgb_clf, tree_index=3, name='Tree4')
graph.render(view=True)

In [ ]:
feature_importance = np.stack([X_train1.columns.values,lgb_clf.feature_importances_]).transpose()
feature_importance = pd.DataFrame(feature_importance,columns=['feature_name','importance'])
feature_importance.sort_values(by=['importance'],inplace=True,ascending=False)
print(feature_importance)

## PCA visualization in one person who has fraud data

In [ ]:
from sklearn.decomposition import PCA
def PCA_plot(x,label):
    x = x.drop(columns=delete_list)
    
    ## 應該先轉dummy,標準化,再PCA
#     dummy_list=['contp','etymd','stscd','hcefg']
#     dummy_list2=['stocn','scity','csmcu']#'mchno','acqic','mcc',
#     x[dummy_list] = x[dummy_list].astype(object)
#     x[dummy_list2] = x[dummy_list2].astype(object)
#     x = pd.get_dummies(x)    
    
    from sklearn.preprocessing import StandardScaler 
    stdsc = StandardScaler() 
    x = stdsc.fit_transform(x)
    print(x.shape,label.sum())

    PCA_model = PCA(n_components=2)
    train_data_pca = PCA_model.fit_transform(x)
    train_data_pca1 = train_data_pca[label==1]
    train_data_pca0 = train_data_pca[label==0]
    
    plt.clf()
    plt.figure(figsize=(10,10))
    plt.scatter(train_data_pca1[:, 0], train_data_pca1[:, 1], c='r',label='fraud transaction',s=100)
    plt.scatter(train_data_pca0[:, 0], train_data_pca0[:, 1], c='b',label='normal transaction',s=3)
    plt.legend()
    plt.show()
    
bacno_hasfraud = all_data[all_data['fraud_ind']==1]['bacno'].unique()
print(bacno_hasfraud.shape[0])
print(all_data[all_data['fraud_ind']==1].shape[0])

for i in range(bacno_hasfraud.shape[0]):
    if all_data[all_data['bacno']==bacno_hasfraud[i]].shape[0]>300:
        print('Ploting PCA on bacno-{}'.format(bacno_hasfraud[i]))
        PCA_plot(all_data[all_data['bacno']==bacno_hasfraud[i]],all_data[all_data['bacno']==bacno_hasfraud[i]]['fraud_ind'])

## TSNE, Kmeans作圖?

## Isolation Forest

In [ ]:
from sklearn.ensemble import IsolationForest

c_ratio = y_train.sum()/y_train.shape[0]
# fit the model
clf = IsolationForest(behaviour='new', max_samples=0.8, max_features=1,
                      random_state=random_seed, contamination=c_ratio)
clf.fit(X_train)

y_pred_train = clf.predict(X_train)
y_pred_test = clf.predict(X_test)

y_pred_test2 = -y_pred_test
y_pred_test2[y_pred_test2==-1]=0
y_pred_test2.sum()

y_pred_train2 = -y_pred_train
y_pred_train2[y_pred_train2==-1]=0
y_pred_train2.sum()

from sklearn.metrics import f1_score
print(f1_score(y_train, y_pred_train2))
print(f1_score(y_test, y_pred_test2))

isolationtree_X_train = clf.score_samples(X_train)
isolationtree_X_test = clf.score_samples(X_test)

print(isolationtree_X_train)

In [ ]:
# cano_lastlocdt
# bacno_lastlocdt
# bacno_cano_nunique



In [ ]:
# bacno_etymd_nunique
# bacno_contp_nunique

In [ ]:
# 用hinge loss(當SVM)

In [ ]:
# X_train['cents']
# encoding data

# GroupKfold
# vanilla KFold